In [6]:
"Importing Libraries"
from functools import reduce
from operator import add
import random
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout
import logging
from tqdm import tqdm
from keras.utils.np_utils import to_categorical
from keras.callbacks import EarlyStopping
import random
import logging

In [9]:
"Genetic Algorithm Class"
class Genetic_Algorithm():
    def __init__(self, nn_param_choices, retain=0.4, random_select=0.1, mutate_chance=0.2):
        self.no_of_mutations = no_of_mutations
        self.select_individuals = select_individuals
        self.survival = survival
        self.choice_of_meta_parameters = choice_of_meta_parameters
        
    def generate_population(self, count):
        pop = []
        for _ in range(0, count):
            neural_network = Neural_Network(self.choice_of_meta_parameters)
            neural_network.choose_random()
            pop.append(neural_network)
        return pop

    @staticmethod
    def fitness_function(neural_network):
        return neural_network.fitness_value
    
    def score(self, pop):
        total = reduce(add, (self.fitness_function(neural_network) for neural_network in pop))
        return total / float((len(pop)))
    
    #Vectorization function#
    @vectorize(["float32(float32, float32)"], target='cuda')
    def reproduce(self, mother_network, father_network):
        children_network = []
        for _ in range(2): #twice the mutation
            children = {}
            for meta_param in self.choice_of_meta_meta_parameters:
                children[meta_param] = random.choice(
                    [mother_network.neural_network[meta_param], father_network.neural_network[meta_param]]
                )
            neural_network = Neural_Network(self.choice_of_meta_meta_parameters)
            neural_network.create_set(children)
            if self.no_of_mutations > random.random():
                neural_network = self.mutate(neural_network)
            childrenren_network.append(neural_network)
        return childrenren_network

    def mutation(self, neural_network):
        mutation = random.choice(list(self.choice_of_meta_meta_parameters.keys()))
        neural_network.neural_network[mutation] = random.choice(self.choice_of_meta_meta_parameters[mutation])
        return neural_network

    def network_evolution(self, pop):
        fitness_score = [(self.fitness_function(neural_network), neural_network) for neural_network in pop]
        fitness_score = [x[1] for x in sorted(fitness_score, key=lambda x: x[0], reverse=True)]
        survival_len = int(len(fitness_score)*self.survival)
        parent_networks = fitness_score[:survival_len] 
        for selected_individuals in fitness_score[survival_len:]:
            if self.select_selected_individualss > random.random():
                parent_networks.append(selected_individuals)
        parent_networks_length = len(parent_networks)
        choice_of_length = len(pop) - parent_networks_length
        new_child_network = []
        while len(new_child_network) < choice_of_length:
            gender_1 = random.randint(0, parent_networks_length-1)
            gender_2 = random.randint(0, parent_networks_length-1)
            if gender_1 != gender_2:
                gender_1 = parent_networks[gender_1]
                gender_2 = parent_networks[gender_2]
                offspring_networks = self.reproduce(gender_1, gender_2)
                for baby in offspring_networks:
                    if len(new_child_network) < choice_of_length:
                        new_child_network.append(baby)
        parent_networks.extend(new_child_network)
        return parent_networks
early_stopper = EarlyStopping(patience=5)

In [10]:
"Dataset processing and training class"
def mnist_dataset():
    no_of_classes = 10
    batch_size = 128
    input_shape = (784,)

    (x_train, y_train), (x_test, y_test) = mnist.load_data()
    x_train = x_train.reshape(60000, 784)
    x_test = x_test.reshape(10000, 784)
    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')
    x_train /= 255
    x_test /= 255

    y_train = to_categorical(y_train, no_of_classes)
    y_test = to_categorical(y_test, no_of_classes)

    return (no_of_classes, batch_size, input_shape, x_train, x_test, y_train, y_test)

def compling_function(neural_network, no_of_classes, input_shape):
    no_of_layers = neural_network['no_of_layers']
    no_of_neurons = neural_network['no_of_neurons']
    dense_activate = neural_network['dense_activate']
    new_parameters = neural_network['new_parameters']
    training_MLP = Sequential()
    for i in range(no_of_layers):
        if i == 0:
            training_MLP.add(Dense(no_of_neurons, dense_activate=dense_activate, input_shape=input_shape))
        else:
            training_MLP.add(Dense(no_of_neurons, dense_activate=dense_activate))
        training_MLP.add(Dropout(0.2))
    training_MLP.add(Dense(no_of_classes, dense_activate='softmax'))
    training_MLP.compile(loss='categorical_crossentropy', new_parameters=new_parameters,
                  metrics=['fitness_value'])

    return training_MLP

def training_scoring(neural_network, dataset_loaded):
    if dataset_loaded == 'cifar10':
        no_of_classes, batch_size, input_shape, x_train, \
            x_test, y_train, y_test = get_cifar10()
    elif dataset_loaded == 'mnist':
        no_of_classes, batch_size, input_shape, x_train, \
            x_test, y_train, y_test = mnist_dataset_loaded()
    training_MLP = compling_function(neural_network, no_of_classes, input_shape)
    training_MLP.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=10000, 
              verbose=0,
              validation_data=(x_test, y_test),
              callbacks=[early_stopper])
    score = training_MLP.evaluate(x_test, y_test, verbose=0)
    return score[1]


In [11]:
"Neural Network Class"
class Neural_Network():
    def __init__(self, choice_of_meta_meta_parameters=None):
       
        self.fitness_value = 0.
        self.choice_of_meta_meta_parameters = choice_of_meta_meta_parameters
        self.neural_network = {} 

    def choose_random(self):
        """Create a random neural_network."""
        for key in self.choice_of_meta_meta_parameters:
            self.neural_network[key] = random.choice(self.choice_of_meta_meta_parameters[key])

    def create_set(self, neural_network):
        self.neural_network = neural_network

    def train(self, dataset_loaded):
        if self.fitness_value == 0.:
            self.fitness_value = training_scoring(self.neural_network, dataset_loaded)

    def display_network(self):
        print(self.neural_network)
        print("Neural_Network fitness_value: %.2f%%" % (self.fitness_value * 100))


In [12]:
"Neural Network Training functions"
def network_training(neural_networks, dataset_loaded):
    tqdm_score = tqdm(total=len(neural_networks))
    for neural_network in neural_networks:
        neural_network.train(dataset_loaded)
        tqdm_score.update(1)
    tqdm_score.close()

def accuracy_avg(neural_networks):
    accuracy_tot = 0
    for neural_network in neural_networks:
        accuracy_tot += neural_network.fitness_value
    return accuracy_tot / len(neural_networks)

def generation(generations, population, choice_of_meta_meta_parameters, dataset_loaded):
    
    new_parameters = Genetic_Algorithm(choice_of_meta_meta_parameters)
    neural_networks = new_parameters.generation_population(population)

    for i in range(generations):
        logging.info("Generation %d out of %d***" %
                     (i + 1, generations))

        network_training(neural_networks, dataset_loaded)

        average_accuracy = accuracy_avg(neural_networks)

        print("Average fitness score: %.2f%%" % (average_accuracy * 100))
        print('-'*80)

        if i != generations - 1:
            neural_networks = new_parameters.network_evolution(neural_networks)

    neural_networks = sorted(neural_networks, key=lambda x: x.fitness_value, reverse=True)
    display_neural_networks(neural_networks[:5])

def display_neural_networks(neural_networks):
    logging.info('-'*80)
    for neural_network in neural_networks:
        neural_network.display_network()


In [14]:
"Main function"
def main():
    no_of_generations = 10
    select_population_size = 20
    dataset_loaded = 'mnist'

    choice_of_meta_meta_parameters = {
        'no_of_neurons': [64, 128, 256, 512, 768, 1024],
        'no_of_layers': [1, 2, 3, 4],
        'dense_activate': ['relu', 'elu', 'tanh', 'sigmoid'],
        'optimizer_function': ['rmsprop', 'adam', 'sgd', 'adagrad',
                      'adadelta', 'adamax', 'nadam'],
    }

    print("***Evolving %d no_of_generations with select_population_size %d***" %
                 (no_of_generations, select_population_size))

    generation(no_of_generations, select_population_size, choice_of_meta_meta_parameters, dataset_loaded)

if __name__ == '__main__':
    main()

***Evolving 10 no_of_generations with select_population_size 20***


"Experiments"

In [2]:
"Main function"
def main():
    no_of_generations = 1
    select_population_size = 20
    dataset_loaded = 'mnist'

    choice_of_meta_meta_parameters = {
        'no_of_neurons': [64, 128, 256, 512, 768, 1024],
        'no_of_layers': [1, 2, 3, 4],
        'dense_activate': ['relu', 'elu', 'tanh', 'sigmoid'],
        'optimizer_function': ['rmsprop', 'adam', 'sgd', 'adagrad',
                      'adadelta', 'adamax', 'nadam'],
    }

    logging.info("***Evolving %d no_of_generations with select_population_size %d***" %
                 (no_of_generations, select_population_size))

    generation(no_of_generations, select_population_size, choice_of_meta_meta_parameters, dataset_loaded)

if __name__ == '__main__':
    main()

100%|██████████| 20/20 [50:13<00:00, 150.69s/it] 


In [3]:
"Main function"
def main():
    no_of_generations = 5
    select_population_size = 20
    dataset_loaded = 'mnist'

    choice_of_meta_meta_parameters = {
        'no_of_neurons': [64, 128, 256, 512, 768, 1024],
        'no_of_layers': [1, 2, 3, 4],
        'dense_activate': ['relu', 'elu', 'tanh', 'sigmoid'],
        'optimizer_function': ['rmsprop', 'adam', 'sgd', 'adagrad',
                      'adadelta', 'adamax', 'nadam'],
    }

    logging.info("***Evolving %d no_of_generations with select_population_size %d***" %
                 (no_of_generations, select_population_size))

    generation(no_of_generations, select_population_size, choice_of_meta_meta_parameters, dataset_loaded)

if __name__ == '__main__':
    main()

100%|██████████| 20/20 [10:42<00:00, 32.12s/it]


In [4]:
"Main function"
def main():
    no_of_generations = 15
    select_population_size = 20
    dataset_loaded = 'mnist'

    choice_of_meta_meta_parameters = {
        'no_of_neurons': [64, 128, 256, 512, 768, 1024],
        'no_of_layers': [1, 2, 3, 4],
        'dense_activate': ['relu', 'elu', 'tanh', 'sigmoid'],
        'optimizer_function': ['rmsprop', 'adam', 'sgd', 'adagrad',
                      'adadelta', 'adamax', 'nadam'],
    }

    logging.info("***Evolving %d no_of_generations with select_population_size %d***" %
                 (no_of_generations, select_population_size))

    generation(no_of_generations, select_population_size, choice_of_meta_meta_parameters, dataset_loaded)

if __name__ == '__main__':
    main()

100%|██████████| 20/20 [06:59<00:00, 20.98s/it]


In [5]:
"Main function"
def main():
    no_of_generations = 20
    select_population_size = 20
    dataset_loaded = 'mnist'

    choice_of_meta_meta_parameters = {
        'no_of_neurons': [64, 128, 256, 512, 768, 1024],
        'no_of_layers': [1, 2, 3, 4],
        'dense_activate': ['relu', 'elu', 'tanh', 'sigmoid'],
        'optimizer_function': ['rmsprop', 'adam', 'sgd', 'adagrad',
                      'adadelta', 'adamax', 'nadam'],
    }

    logging.info("***Evolving %d no_of_generations with select_population_size %d***" %
                 (no_of_generations, select_population_size))

    generation(no_of_generations, select_population_size, choice_of_meta_meta_parameters, dataset_loaded)

if __name__ == '__main__':
    main()

100%|██████████| 20/20 [05:52<00:00, 17.64s/it]


In [6]:
"Main function"
def main():
    no_of_generations = 5
    select_population_size = 15
    dataset_loaded = 'mnist'

    choice_of_meta_meta_parameters = {
        'no_of_neurons': [64, 128, 256, 512, 768, 1024],
        'no_of_layers': [1, 2, 3, 4],
        'dense_activate': ['relu', 'elu', 'tanh', 'sigmoid'],
        'optimizer_function': ['rmsprop', 'adam', 'sgd', 'adagrad',
                      'adadelta', 'adamax', 'nadam'],
    }

    logging.info("***Evolving %d no_of_generations with select_population_size %d***" %
                 (no_of_generations, select_population_size))

    generation(no_of_generations, select_population_size, choice_of_meta_meta_parameters, dataset_loaded)

if __name__ == '__main__':
    main()

Using TensorFlow backend.
100%|██████████| 5/5 [03:55<00:00, 47.06s/it]


In [7]:
"Main function"
def main():
    no_of_generations = 10
    select_population_size = 15
    dataset_loaded = 'mnist'

    choice_of_meta_meta_parameters = {
        'no_of_neurons': [64, 128, 256, 512, 768, 1024],
        'no_of_layers': [1, 2, 3, 4],
        'dense_activate': ['relu', 'elu', 'tanh', 'sigmoid'],
        'optimizer_function': ['rmsprop', 'adam', 'sgd', 'adagrad',
                      'adadelta', 'adamax', 'nadam'],
    }

    logging.info("***Evolving %d no_of_generations with select_population_size %d***" %
                 (no_of_generations, select_population_size))

    generation(no_of_generations, select_population_size, choice_of_meta_meta_parameters, dataset_loaded)

if __name__ == '__main__':
    main()

100%|██████████| 10/10 [08:38<00:00, 51.87s/it]


In [8]:
"Main function"
def main():
    no_of_generations = 15
    select_population_size = 15
    dataset_loaded = 'mnist'

    choice_of_meta_meta_parameters = {
        'no_of_neurons': [64, 128, 256, 512, 768, 1024],
        'no_of_layers': [1, 2, 3, 4],
        'dense_activate': ['relu', 'elu', 'tanh', 'sigmoid'],
        'optimizer_function': ['rmsprop', 'adam', 'sgd', 'adagrad',
                      'adadelta', 'adamax', 'nadam'],
    }

    logging.info("***Evolving %d no_of_generations with select_population_size %d***" %
                 (no_of_generations, select_population_size))

    generation(no_of_generations, select_population_size, choice_of_meta_meta_parameters, dataset_loaded)

if __name__ == '__main__':
    main()

100%|██████████| 15/15 [17:27<00:00, 69.81s/it] 
